## TuneTA Research

In [1]:
import os
from pathlib import Path


# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/Users/liuqiaowei/SourceCode/freqtrade/user_data/notebooks"
i = 0
try:
    os.chdir(project_root)
    if not Path("LICENSE").is_file():
        i = 0
        while i < 4 and (not Path("LICENSE").is_file()):
            os.chdir(Path(Path.cwd(), "../"))
            i += 1
        project_root = Path.cwd()
except FileNotFoundError:
    print("Please define the project root relative to the current directory")
print(Path.cwd())

/Users/liuqiaowei/SourceCode/freqtrade


In [2]:
from freqtrade.configuration import Configuration


# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["./user_data/config.json"])
# Optionally (recommended), use existing configuration file
# config = Configuration.from_files(["user_data/config.json"])

# Location of the data
data_location = config["datadir"]
config['timeframe'] = '1h'
# Pair to analyze - Only use one pair here
# config['exchange']['pair_whitelist']

In [4]:
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from pandas_ta import percent_return

candles_dict = {}
for pair in config['exchange']['pair_whitelist']:
    candles = load_pair_history(
        datadir=data_location,
        timeframe=config["timeframe"],
        pair=pair,
        data_format="feather",  # Make sure to update this to your data
        candle_type=CandleType.FUTURES,
    )
    candles['sym'] = pair
    candles.set_index('date', inplace=True)
    candles.set_index('sym', append=True, inplace=True)
    candles['return'] = percent_return(candles.close, offset=-1)
    # Keep Latest week for testing
    candles_dict[pair] = candles[-24 * 30:]


In [5]:
candles_dict['DASH/USDT:USDT']

,,open,high,low,close,volume,return
date,sym,,,,,,
2024-10-20 04:00:00+00:00,DASH/USDT:USDT,23.94,24.02,23.87,23.96,4762.700,-0.001669
2024-10-20 05:00:00+00:00,DASH/USDT:USDT,23.97,24.03,23.91,23.92,5079.337,0.006271
2024-10-20 06:00:00+00:00,DASH/USDT:USDT,23.93,24.09,23.88,24.07,4905.979,0.000000
2024-10-20 07:00:00+00:00,DASH/USDT:USDT,24.07,24.14,24.03,24.07,6301.549,0.003324
2024-10-20 08:00:00+00:00,DASH/USDT:USDT,24.07,24.29,24.07,24.15,9956.549,0.002484
...,...,...,...,...,...,...,...
2024-11-18 23:00:00+00:00,DASH/USDT:USDT,29.54,29.68,29.22,29.62,9703.122,0.003038
2024-11-19 00:00:00+00:00,DASH/USDT:USDT,29.65,29.83,29.46,29.71,13340.689,-0.008751
2024-11-19 01:00:00+00:00,DASH/USDT:USDT,29.71,29.84,29.37,29.45,11256.740,0.003396


In [6]:
from tuneta.tune_ta import TuneTA
import pandas as pd

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [7]:
X = pd.concat([candles for _, candles in candles_dict.items()], axis=0).sort_index()
X.dropna(inplace=True)
y = X['return']
X = X.drop(columns=['return'])

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=False)

In [9]:
tt = TuneTA(n_jobs=4, verbose=True)

In [11]:
tt.fit(X_train, y_train,
        indicators=['tta.ADX', 'tta.ADXR', 'tta.APO', 'tta.AROON', 'tta.AROONOSC', 'tta.BOP', 'tta.CCI', 'tta.CMO', 'tta.DX', 'tta.MACD', 'tta.MACDEXT', 'tta.MACDFIX', 'tta.MFI', 'tta.MINUS_DI', 'tta.MINUS_DM', 'tta.MOM', 'tta.PLUS_DI', 'tta.PLUS_DM', 'tta.PPO', 'tta.ROC', 'tta.ROCP', 'tta.ROCR', 'tta.ROCR100', 'tta.RSI', 'tta.STOCH', 'tta.STOCHF', 'tta.STOCHRSI', 'tta.TRIX', 'tta.ULTOSC','tta.WILLR', 'tta.AD', 'tta.ADOSC', 'tta.OBV', 'tta.ATR', 'tta.NATR', 'tta.TRANGE'],
        ranges=[(4, 12), (13, 24)],
        trials=100,
        early_stop=10,
)
  
tt.fit_times()

    Indicator       Times
--  ------------  -------
19  tta.STOCHRSI    14.31
68  tta.MACD        13.5
21  tta.STOCHRSI    12.85
62  tta.MACD        12.25
59  tta.MACDEXT     12.01
29  tta.ULTOSC      11.87
24  tta.CCI         11.35
34  tta.DX           9.72
37  tta.STOCHF       9.7
31  tta.DX           9.7
57  tta.STOCH        9.34
63  tta.MACDEXT      9.26
30  tta.AROON        9.23
47  tta.ADOSC        9.22
33  tta.ULTOSC       9.2
32  tta.MFI          9.16
54  tta.ADOSC        9.07
 5  tta.ROC          8.71
 6  tta.ROCR100      8.65
 3  tta.ROCR         8.44
60  tta.APO          8.34
 4  tta.ROCP         8.32
40  tta.MFI          7.35
38  tta.WILLR        7.34
43  tta.ATR          7.16
10  tta.ROCP         7.07
 8  tta.ROCR         7.01
27  tta.CMO          7.01
15  tta.MINUS_DI     7
58  tta.ADX          6.98
 0  tta.NATR         6.94
 7  tta.ROCR100      6.94
64  tta.MACDFIX      6.93
66  tta.MOM          6.92
18  tta.STOCH        6.91
70  tta.ADXR         6.89
 9  tta.ROC        

In [12]:
tt.report(target_corr=True, features_corr=False)


Indicator Correlation to Target:

                                                              Correlation
----------------------------------------------------------  -------------
tta_NATR_timeperiod_7                                            0.165787
tta_NATR_timeperiod_16                                           0.162139
tta_PPO_fastperiod_17_slowperiod_24                              0.083103
tta_ROCR_timeperiod_21                                           0.083005
tta_ROCP_timeperiod_21                                           0.083005
tta_ROC_timeperiod_21                                            0.083005
tta_ROCR100_timeperiod_21                                        0.083005
tta_ROCP_timeperiod_7                                            0.080359
tta_ROC_timeperiod_7                                             0.080359
tta_ROCR_timeperiod_7                                            0.080359
tta_ROCR100_timeperiod_7                                         0.080359
tta

In [13]:
tt.prune(max_inter_correlation=.8)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [14]:
tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                              Correlation
----------------------------------------------------------  -------------
tta_NATR_timeperiod_7                                            0.165787
tta_PPO_fastperiod_17_slowperiod_24                              0.083103
tta_ROCR100_timeperiod_7                                         0.080359
tta_TRIX_timeperiod_17                                           0.079961
tta_MINUS_DI_timeperiod_21                                       0.070565
tta_MINUS_DI_timeperiod_7                                        0.069703
tta_BOP                                                          0.069159
tta_PLUS_DI_timeperiod_20                                        0.062903
tta_STOCHRSI_timeperiod_11_fastk_period_11_fastd_period_11       0.05965
tta_AROONOSC_timeperiod_22                                       0.051144
tta_ULTOSC_timeperiod1_23_timeperiod2_21_timeperiod3_23          0.04921
tta_A

In [17]:
tt.t_corr[tt.t_corr['Correlation'] > 0.03]

,Correlation
tta_NATR_timeperiod_7,0.165787
tta_PPO_fastperiod_17_slowperiod_24,0.083103
tta_ROCR100_timeperiod_7,0.080359
tta_TRIX_timeperiod_17,0.079961
tta_MINUS_DI_timeperiod_21,0.070565
tta_MINUS_DI_timeperiod_7,0.069703
tta_BOP,0.069159
tta_PLUS_DI_timeperiod_20,0.062903
tta_STOCHRSI_timeperiod_11_fastk_period_11_fastd_period_11,0.059650
tta_AROONOSC_timeperiod_22,0.051144


In [16]:
tt.t_corr.to_csv('./tune_ta_result.csv')

In [ ]:
features = tt.transform(X_train)

In [ ]:
features.columns